In [1]:
# import pandas as pd
import sys
import pandas as pd
import altair as alt
from utils.io import get_data
from data_prep import prep_dataframe

start_date = 20200101
end_date = 20240101
fname = f"{start_date}_{end_date}"
download_filepath = f"./data/in/{fname}"



In [2]:

df = get_data(start_date, end_date, download_filepath, cache=True)
if df is None or df.empty:
  print("ERROR: No data")
  sys.exit(1)


INFO: Using cached data


In [3]:
df = prep_dataframe(df)
df.info()

fixing PDCMethodOfUse Ingests to Ingest
fixing PDCMethodOfUse Injects to Inject
fixing PDCMethodOfUse Smokes to Smoke
<class 'pandas.core.frame.DataFrame'>
Index: 7317 entries, 6973 to 1519
Columns: 186 entries, SLK to PDCGoals
dtypes: category(17), datetime64[ns](1), float64(11), object(157)
memory usage: 9.6+ MB


In [21]:
# pq_df.iloc[:, :10].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7318 entries, 0 to 7317
Data columns (total 10 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   PartitionKey                                 7318 non-null   object        
 1   RowKey                                       7318 non-null   object        
 2   SurveyName                                   7318 non-null   object        
 3   Program                                      2064 non-null   category      
 4   AssessmentDate                               7318 non-null   datetime64[ns]
 5   Anyodc                                       7278 non-null   object        
 6   Past4WkNumInjectingDays                      1683 non-null   object        
 7   HaveYouEverSharedEquipment                   3035 non-null   object        
 8   Past4WkDailyLivingImpacted                   6374 non-null   object        
 9

In [3]:
pq_df.iloc[:, 10:20].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7318 entries, 0 to 7317
Data columns (total 10 columns):
 #   Column                                     Non-Null Count  Dtype   
---  ------                                     --------------  -----   
 0   Past4WkHowOftenMentalHealthCausedProblems  6381 non-null   category
 1   Past4WkUseLedToProblemsWithFamilyFriend    6375 non-null   category
 2   Past4WkHowOftenIllegalActivities           6225 non-null   category
 3   HowCloseToManagingSubstanceUse             6381 non-null   object  
 4   HowSatisfiedWithProgress                   6083 non-null   object  
 5   Past4WkDifficultyFindingHousing            6285 non-null   object  
 6   Staff                                      1032 non-null   object  
 7   ClientType                                 6286 non-null   category
 8   IndigenousStatus                           3374 non-null   category
 9   SDSIsAODUseOutOfControl                    5924 non-null   object  
dtypes: category(

In [5]:
pq_df.Past4WkHowOftenIllegalActivities.unique()

['Not at all', NaN, 'Less than weekly', 'Three or four times per week', 'Daily or almost daily', 'Once or twice per week']
Categories (5, object): ['Not at all' < 'Less than weekly' < 'Once or twice per week' < 'Three or four times per week' < 'Daily or almost daily']

In [4]:
pqpdc = df[df.PDCSubstanceOrGambling.notna()]
len(pqpdc) , len(df)
          # pq_df.columns.str.contains('PDC', case=True)]

(7317, 7318)

In [7]:
pqpdc.Program.value_counts(dropna=False)

Program
TSS          2647
EUROPATH      918
GOLBICE       729
MURMICE       641
GOLBGNRL      475
SAPPHIRE      456
BEGAPATH      374
MURMWIO       311
MURMPP        299
COCO          154
MONPATH       143
ARCA           88
MURMHEAD       62
ALONGSIDE      19
PSYNSW          1
Name: count, dtype: int64

In [17]:
# pqpdc['Program'] = pqpdc['RowKey'].str.split('_').str[0]

C:\Users\aftab.jalal\AppData\Local\Temp\ipykernel_12204\377265289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pqpdc['Program'] = pqpdc['RowKey'].str.split('_').str[0]


In [8]:
from utils.group_utils import  limit_pkey_num_assessments, get_clean_for_col
# ESSENTIAL_FIELDS = ['PartitionKey', 'RowKey', 'AssessmentDate']
pkeys_inperiod = pqpdc[ (pqpdc.AssessmentDate >=  '2022-07-01') & (pqpdc.AssessmentDate <= '2023-06-30')].PartitionKey.unique()
pqpdc = pqpdc [ pqpdc['PartitionKey'].isin(pkeys_inperiod) ]

min_assessments_per_client = 3

dfmin3 = limit_pkey_num_assessments(pqpdc, min_assessments_per_client)

col ='SDS_Score'
col_df = get_clean_for_col(dfmin3, col)
len(pqpdc), min(pqpdc.AssessmentDate), max(pqpdc.AssessmentDate)

(3745, Timestamp('2020-01-02 00:00:00'), Timestamp('2023-06-22 00:00:00'))

In [1]:
len(df), len(pqpdc) ,len(dfmin3), len(col_df)

NameError: name 'df' is not defined

In [10]:
df_period = pqpdc.sort_values(['AssessmentDate'])

In [25]:

# from pandas.api.types import CategoricalDtype
# category_notatall_to_daily = CategoricalDtype(['Not at all',  'Less than weekly', 'Once or twice per week',
#        'Three or four times per week', 'Daily or almost daily'], ordered=True)

# category_notatall_to_daily.names

In [11]:
# o = df['Past4WkHowOftenPhysicalHealthCausedProblems'].value_counts(dropna=False)
# NaN                              936

# Drop the rows where Past4WkHowOftenPhysicalHealthCausedProblems is NaN
df_phy = df_period.dropna(subset=['Past4WkHowOftenPhysicalHealthCausedProblems'])

# df['physical'] = pd.Categorical(df['Past4WkHowOftenPhysicalHealthCausedProblems'], ordered=True, categories=[0, 1, 2, 3, 4])

# # Convert the categorical column to an integer type
# df_phy['Past4WkHowOftenPhysicalHealthCausedProblems'] = df_phy['Past4WkHowOftenPhysicalHealthCausedProblems'].astype(int)
# df['Past4WkHowOftenPhysicalHealthCausedProblems'] = 
# df['Past4WkHowOftenPhysicalHealthCausedProblems'].cat.codes, df['Past4WkHowOftenPhysicalHealthCausedProblems'].cat.categories

# p = df['Past4WkHowOftenPhysicalHealthCausedProblems'].value_counts()
# o,p 

In [12]:
df_phy['Past4WkHowOftenPhysicalHealthCausedProblems'].value_counts(dropna=False)

Past4WkHowOftenPhysicalHealthCausedProblems
Not at all                      1654
Daily or almost daily            691
Less than weekly                 583
Once or twice per week           407
Three or four times per week     264
Name: count, dtype: int64

In [10]:
# df = df.dropna(subset=['Past4WkHowOftenPhysicalHealthCausedProblems'])
df_phy['Past4WkHowOftenPhysicalHealthCausedProblems'].cat.codes.value_counts(dropna=True)

0    1654
4     691
1     583
2     407
3     264
Name: count, dtype: int64

In [13]:
# Assuming 'PartitionKey' is the patient identifier and 'AssessmentDate' is in datetime format
df_sorted = df_phy.sort_values(by=['PartitionKey', 'AssessmentDate'])

g = df_sorted.groupby('PartitionKey')

# Get the first and last assessments
first_assessments = g.first().reset_index()
last_assessments = g.last().reset_index()




In [14]:
len(g), len(first_assessments), len(last_assessments)

(1510, 1510, 1510)

In [14]:

# Calculate the mean for the first and last assessments
mean_first = first_assessments['Past4WkHowOftenPhysicalHealthCausedProblems'].cat.codes.mean()
mean_last = last_assessments['Past4WkHowOftenPhysicalHealthCausedProblems'].cat.codes.mean()

# Create a DataFrame for the mean values
mean_df = pd.DataFrame({
    'Assessment': ['First', 'Last'],
    'MeanValue': [mean_first, mean_last]
})

# # Create a line plot
# chart = alt.Chart(mean_df).mark_line(point=True).encode(
#     x='Assessment',
#     y='MeanValue',
#     tooltip=['Assessment', 'MeanValue']
# ).properties(
#     title='Mean Values of First and Last Assessments'
# )

# Create a date range
date = pd.date_range(start='01-01-2020', end='01-01-2023')

# Create a sample DataFrame
df = pd.DataFrame({
  'date': np.repeat(date, 3),
  'category': ['A', 'B', 'C']*len(date),
  'value': np.random.rand(3*len(date)),
})

chart = alt.Chart(df).mark_line().encode(
    x='date:T',
    y='value:Q',
    color='category:N',
    tooltip=['date', 'value', 'category']
).properties(
    title='Multi-Line Chart'
)


# Display the chart
chart


alt.Chart(...)

In [ ]:
# Calculate the mean for the first and last assessments for both variables
mean_first_physical = first_assessments['Past4WkHowOftenPhysicalHealthCausedProblems'].cat.codes.mean()
mean_last_physical = last_assessments['Past4WkHowOftenPhysicalHealthCausedProblems'].cat.codes.mean()
mean_first_mental = first_assessments['Past4WkHowOftenMentalHealthCausedProblems'].cat.codes.mean()
mean_last_mental = last_assessments['Past4WkHowOftenMentalHealthCausedProblems'].cat.codes.mean()


In [ ]:

# Create DataFrames for the mean values
mean_physical_df = pd.DataFrame({
    'Assessment': ['First', 'Last'],
    'MeanValue': [mean_first_physical, mean_last_physical]
})

mean_mental_df = pd.DataFrame({
    'Assessment': ['First', 'Last'],
    'MeanValue': [mean_first_mental, mean_last_mental]
})

# Create the line plots
chart_physical = alt.Chart(mean_physical_df).mark_line(point=True).encode(
    x='Assessment',
    y='MeanValue',
    tooltip=['Assessment', 'MeanValue']
).properties(
    title='Mean Values of First and Last Physical Health Assessments'
)

chart_mental = alt.Chart(mean_mental_df).mark_line(point=True).encode(
    x='Assessment',
    y='MeanValue',
    tooltip=['Assessment', 'MeanValue']
).properties(
    title='Mean Values of First and Last Mental Health Assessments'
)

# Concatenate the charts vertically
chart = alt.hconcat(chart_physical, chart_mental)

# Display the chart
chart


In [40]:
first_fours_phy = len(first_assessments['Past4WkHowOftenPhysicalHealthCausedProblems'] [ first_assessments['Past4WkHowOftenPhysicalHealthCausedProblems'].cat.codes == 4])
last_fours_phy = len(last_assessments['Past4WkHowOftenPhysicalHealthCausedProblems'] [ last_assessments['Past4WkHowOftenPhysicalHealthCausedProblems'].cat.codes == 4])
first_fours_men =len( first_assessments['Past4WkHowOftenMentalHealthCausedProblems'] [ first_assessments['Past4WkHowOftenMentalHealthCausedProblems'].cat.codes == 4])
last_fours_men = len(last_assessments['Past4WkHowOftenMentalHealthCausedProblems'] [ last_assessments['Past4WkHowOftenMentalHealthCausedProblems'].cat.codes == 4])

# first_fours_phy, last_fours_phy, first_fours_men, last_fours_men

# percentage change in 'Daily or almost daily' 
phy_pc = (last_fours_phy - first_fours_phy)* 100 / first_fours_phy
men_pc = (last_fours_men - first_fours_men)* 100 / first_fours_men

physicalhealth_problems_change = round(phy_pc,1)
mentallhealth_problems_change = round(men_pc,1)

In [45]:


# Create a DataFrame with 'category' and 'mean_value' columns
mean_df = pd.DataFrame({
    'Assessment': ['First', 'Last', 'First', 'Last'],
    'MeanValue': [mean_first_physical, mean_last_physical, mean_first_mental, mean_last_mental],
    'Category': ['Physical Health']*2 + ['Mental Health']*2,
    'Count': [len(first_assessments), len(last_assessments), len(first_assessments), len(last_assessments)]
})

# Create the line plot
chart = alt.Chart(mean_df).mark_line().encode(
    x='Assessment',
    y='MeanValue',
    color=alt.Color('Category', scale=alt.Scale(domain=['Physical Health', 'Mental Health'], range=['red', 'blue'])),
    tooltip=['Assessment', 'MeanValue', 'Category']
).properties(
    title='Mean Values of First and Last Assessments'
).properties(
    width=600  # Specify the width here
)

# Add points to the line plot
points = alt.Chart(mean_df).mark_point(size=100).encode(
    x='Assessment',
    y='MeanValue',
    color=alt.Color('Category', scale=alt.Scale(domain=['Physical Health', 'Mental Health'], range=['red', 'blue'])),
    tooltip=['Assessment', 'MeanValue', 'Category', 'Count']
).properties(
    width=600  # Specify the width here
)



# Define text annotations
annotations = pd.DataFrame({
    'Assessment': ['First', 'First'],
    'MeanValue': [mean_first_physical, mean_first_mental],  # You can adjust the y position
    'Text': [f"% change in number of 'daily or almost daily': {physicalhealth_problems_change}%" ,
              f"% change in number of 'daily or almost daily': {mentallhealth_problems_change}%"]  # Replace with your actual text
})

# Create the text chart
text_chart = alt.Chart(annotations).mark_text(
    align='left',
    baseline='middle',
    dx=7  # Nudges text to right so it doesn't appear on top of the point
).encode(
    x='Assessment',
    y='MeanValue:Q',
    text='Text'
)


# Display the chart
chart = chart + points + text_chart
chart


alt.LayerChart(...)